In [ ]:
import numpy as np
import tensorflow as tf

class two_step_mdp():
    def __init__(self, tranprob = 0.75, rewprob = 0.9):
        self.state_space = (0,)
        self.action_space = (0, 1)
        self.COMMON = True
        self.RARE = False
        self.state = 0
        self.tran_prob = tranprob
        self.rew_prob = rewprob
        self.transition = {0: lambda: (1, self.COMMON) if np.random.rand(1) < self.tran_prob else (2, self.RARE),
                    1: lambda: (2, self.COMMON) if np.random.rand(1) < self.tran_prob else (1, self.RARE)}
        self.reward = {1: lambda: 1 if np.random.rand(1) < self.rew_prob else 0,
                    2: lambda: 1 if np.random.rand(1) < 1 - self.rew_prob else 0}
        self.original = True
        #self.switch()
        
    def get_state(self):
        return self.state
    
    def get_reward(self):
        return self.reward[self.state]()
    
    def step(self, action):
        self.state, common = self.transition[action]()
        return self.state, self.reward[self.state](), common
    
    def rand_step(self):
        #self.state = self.transition[((np.random.random(1)<0.5)[0])]()
        #return self.state, self.reward[self.state]()
        return (np.random.random(1)<0.5)[0]

    def reset(self):
        self.state = 0
        return self.state
    
    def switch(self):
        self.rew_prob = (1-self.rew_prob) if (np.random.random(1)<0.5)[0] else self.rew_prob
        self.original = not self.original
        return self.original


num_trials = 10
num_episodes = 20

inptSt = tf.placeholder(dtype=tf.int32)
oneH = tf.one_hot(inptSt,3)
Q = tf.Variable(tf.random_uniform([3,2],0,0.01))
qVals= tf.matmul([oneH],Q)
outAct= tf.argmax(qVals,1)

nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - qVals))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateMod = trainer.minimize(loss)
init = tf.global_variables_initializer()

gamma = 1
game = two_step_mdp()
rTot=0
stayNumCommReward=0
stayNumCommNoReward=0
stayNumRareReward=0
stayNumRareNoReward=0
numCommRewarded=0
numCommNotRewarded=0
numRareRewarded=0
numRareNotRewarded=0

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        game.switch()
        for j in range(num_trials):
            e = 0.05/(i + 0.05)
            s=game.reset()
            nActs, nxtQ=sess.run([outAct,qVals],feed_dict={inptSt: s})
            if i>=1:
                if rwd:      
                    if comm:
                        stayNumCommReward += 1 if nActs[0] == nAct else 0
                        numCommRewarded += 1
                    else:
                        stayNumRareReward += 1 if nActs[0] == nAct else 0
                        numRareRewarded += 1
                else:
                    if comm:
                        stayNumCommNoReward += 1 if nActs[0] == nAct else 0
                        numCommNotRewarded += 1
                    else:
                        stayNumRareNoReward += 1 if nActs[0] == nAct else 0
                        numRareNotRewarded += 1
            print(nActs[0]==nAct)
            nAct=nActs[0]
            if i<20 and np.random.rand(1)<e: nAct= game.rand_step()
            s1, rwd, comm = game.step(nAct)
            Q1 = sess.run(qVals,feed_dict={inptSt: s1})
            print(nxtQ)
            nxtQ[0,nAct] = rwd + gamma*(np.max(Q1))
            sess.run(updateMod,feed_dict={inptSt:s, nextQ:nxtQ})
            rTot+=rwd
        print("-----------")

#print "Fraction of games succesful: ", rTot/2000.0

print stayNumCommReward/(1.0*numCommRewarded)
print stayNumRareReward/(1.0*numRareRewarded)
print stayNumCommNoReward/(1.0*numCommNotRewarded)
print stayNumRareNoReward/(1.0*numRareNotRewarded)

print stayNumCommReward
print stayNumRareReward
print stayNumCommNoReward
print stayNumRareNoReward

print numCommRewarded
print numRareRewarded
print numCommNotRewarded
print numRareNotRewarded